# Train YOLOv5 (and v8 soon!) on Weed-AI Datasets

This guide will take you through training a state-of-the-art object detection architecture - YOLOv5 - on Weed-AI datasets.

**Steps:**
1. Setup the project: creating folders, cloning YOLOv5
2. Download the Weed-AI dataset
3. Convert weedCOCO to YOLO annotation format
4. Create YOLOv5 supporting files
5. Train YOLOv5
6. Inference on pictures/videos

The tutorial requires you to have access to a Google Drive account and be able to upload images/data to specific folders. Algorithms will train fastest with a GPU. Select the GPU type under 'Runtime' > 'Change Runtime Type'. Make sure GPU is selected. Premium or High RAM will improve speed/size of models that can be trained.



In [ ]:
# mount google drive - this gives the Colab notebook access to your Drive. It may ask you for permission/to sign in too.
from google.colab import drive
drive.mount('/content/drive')

# Create Project Folder

To begin, create a project folder in your Google Drive. We'll call this one `weedai_yolo`. Replace this with whatever name you decide. 

It will be created in the root folder of your Google Drive. In this folder we'll be cloning the [YOLOv5 GitHub Repository](https://github.com/ultralytics/yolov5) and saving our data too. There are many guides on training YOLOv5 that are accessible through the official repository, make sure to check those for any tips/tricks on tuning your model.

In [ ]:
YOUR_DIRECTORY = 'weedai_yolo'

!mkdir /content/drive/MyDrive/{YOUR_DIRECTORY}
%ls '/content/drive/MyDrive/' # should list everything in your Google Drive - double check that your project folder is there.

**(first time only)**

Clone the YOLOv5 repository so we can use it to train our models. Only do this ONCE at the start of the project.

In [ ]:
%cd /content/drive/MyDrive/{YOUR_DIRECTORY}
!git clone https://github.com/ultralytics/yolov5 # clone the YOLOv5 repository. It is a large repository and may take some time depending on your internet speed.

# Downloading a Weed-AI dataset

For this example I've used the [Northern WA Wheatbelt Blue Lupins](https://weed-ai.sydney.edu.au/datasets/9df290f4-a29b-44b2-9de6-24bca1cee846) dataset but any of the other object detection datasets would work too, including the recrntly uploaded [Amsinckia in chickpeas](https://weed-ai.sydney.edu.au/datasets/21675efe-9d25-4096-be76-3a541475efd4) dataset. 

Download the dataset. Then, we'll create a datasets folder in the YOLOv5 directory and then move the Weed-AI dataset to that folder. 

To summarise, the steps we will follow below are:
1. Download the dataset on Weed-AI by clicking the button 'Download in WEEDCOCO format'
2. Unzip the download and move it into the Google Drive Yolov5 folder where your project has been created. For me, this is the `'weedai_yolo/yolov5/datasets'`
3. Convert it from WeedCOCO to YOLOv5 format

Assuming you've downloaded the dataset and unzipped it, follow the steps below.

In [ ]:
YOUR_DATASET = 'blue_lupins' # set the name of your dataset folder here. It should match the name of the folder with images and the weedcoco.json



Create the dataset folder where you'll move the unzipped folder renamed to `blue_lupins` to.

In [ ]:
!mkdir /content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/datasets

Once the dataset has downloaded and is in the datasets folder, it should have a similar structure to the following:
* yolov5/datasets
    * blue_lupins
        * images
        * weedcoco.json


# Convert weedCOCO to YOLO

The first step in the process is converting the downloaded weedCOCO dataset into the YOLO .txt format. The method below is adapted from the official [Ultralytics GitHub repository](https://github.com/ultralytics/JSON2YOLO/blob/master/labelbox_json2yolo.py). Don't worry too much about the code, just press 'Play' on each cell.

In [ ]:
import os
from pathlib import Path

import yaml
import shutil
from tqdm import tqdm
import contextlib
import json

import pandas as pd
import numpy as np
from PIL import Image
from collections import defaultdict

def make_dirs(dir='new_dir/'):
    # Create folders
    dir = Path(dir)
    for p in dir, dir / 'labels', dir / 'images':
        p.mkdir(parents=True, exist_ok=True)  # make dir
    return dir


def convert_weedcoco_json(json_dir=''):
    save_dir = make_dirs(dir=f'{json_dir}')  # output directory
    print()

    # Import json
    for json_file in sorted(Path(json_dir).resolve().glob('*.json')):
        fn = Path(save_dir) # / 'labels' # folder name
        fn.mkdir(exist_ok=True)
        with open(json_file) as f:
            data = json.load(f)

        # Create image dict
        images = {'%g' % x['id']: x for x in data['images']}
        # Create image-annotations dict
        imgToAnns = defaultdict(list)
        for ann in data['annotations']:
            imgToAnns[ann['image_id']].append(ann)
 

        # Write labels file
        for img_id, anns in tqdm(imgToAnns.items(), desc=f'Annotations {json_file}'):
            # print(img_id, anns)
            img = images['%g' % img_id]
            h, w, f = img['height'], img['width'], img['file_name']

            bboxes = []
            segments = []
            for ann in anns:
                # The COCO box format is [top left x, top left y, width, height]
                box = np.array(ann['bbox'], dtype=np.float64)
                box[:2] += box[2:] / 2  # xy top-left corner to center
                box[[0, 2]] /= w  # normalize x
                box[[1, 3]] /= h  # normalize y
                if box[2] <= 0 or box[3] <= 0:  # if w <= 0 and h <= 0
                    continue

                cls = ann['category_id']  # class
                box = [cls] + box.tolist()
                if box not in bboxes:
                    bboxes.append(box)

            # Write
            with open((fn / f.replace('images', 'labels')).with_suffix('.txt'), 'a') as file:
                for i in range(len(bboxes)):
                    line = *(bboxes[i]),  # cls, box or segments
                    file.write(('%g ' * len(line)).rstrip() % line + '\n')

    # Save dataset.yaml
    names = [data['categories'][i]['name'].split(': ')[1] for i in range(len(data['categories']))]
    d = {'path': json_dir,
         'train': 'images/train',
         'val': 'images/train',
         'test': 'images/train',
         'nc': len(names),
         'names': names}  # dictionary

    with open(f"{save_dir}/weedcoco.yaml", 'w') as f:
        yaml.dump(d, f, sort_keys=False)


    print('\nConversion completed successfully!')


In [ ]:
WEED_COCO_LOCATION = f"/content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/datasets/{YOUR_DATASET}"
#convert the weedcoco file
convert_weedcoco_json(json_dir=WEED_COCO_LOCATION)

## Splitting the dataset into train/validation/test
An algorithm needs a training portion and a validation portion to check as it learns. The test portion is left entirely unseen and can be used later to make sure the algorithm hasn't just overfit. 

If you find the algorithm performs well on the training data but terribly on the val/test data, then it is likely overfitting. This is more common on small datasets and larger models when trained for many epochs.

In [ ]:
from sklearn.model_selection import train_test_split

# Read images and annotations
images = [os.path.join(f'{WEED_COCO_LOCATION}/images', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/images')]
annotations = [os.path.join(f'{WEED_COCO_LOCATION}/labels', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/labels') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-valid-test splits 80-10-10%
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

%cd {WEED_COCO_LOCATION}
!mkdir images/train images/val images/test labels/train labels/val labels/test

In [ ]:
#Utility function to move images 
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')
move_files_to_folder(test_annotations, 'labels/test/')

In [ ]:
# Check the images have been moved
print(len(os.listdir('images/train')), len(os.listdir('labels/train')))
print(len(os.listdir('images/val')), len(os.listdir('labels/val')))
print(len(os.listdir('images/test')), len(os.listdir('labels/test')))

# Preparing for training
Now we have all the splits made, we need to import some packages and install other YOLOv5 requirements before we can start.

In [ ]:
# import necessary packages
import torch
from IPython.display import Image  # for displaying images
import os 
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(0)

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

In [ ]:
%cd /content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5
!pip install -r requirements.txt

In [ ]:
# Weights & Biases  (optional)
%pip install -q wandb
import wandb
wandb.login()

# YOLOv5 Training

No we can train a model! Change the name of your run to whatever you like, and try playing around with things like image size, batch size, epochs and YOLOv5 variant. Larger variants and larger images will probably do better, but require more memory. So if you run out of memory, just reduce image size or model variant size (choose M instead of X) and then try again.

Information on selecting batch size: https://twitter.com/rasbt/status/1617544195220312066


In [ ]:
# train YOLOv5m
BATCH = 8
EPOCHS = 30
IMAGE_SIZE = 1280 # (should be one of 320, 640, 1280, 1920)
MODEL = 'm'

# this is the name of your run, and how it will be saved
RUN_NAME = f'{YOUR_DATASET}_TRAIN_B{str(BATCH)}_E{str(EPOCHS)}_SZ{str(IMAGE_SIZE)}_M{MODEL}'

# avoid making any changes to the below, or check the Ultralytics docs for other commands
!python train.py --img {IMAGE_SIZE} --cfg yolov5{MODEL}.yaml --batch {BATCH} --epochs {EPOCHS} --data datasets/{YOUR_DATASET}/weedcoco.yaml --weights yolov5{MODEL}.pt --name {RUN_NAME}

# Detect
This is where you can run the model you've just trained on a sample video or other dataset to see how it goes. The --source flag below accepts videos, folders of images and images. All you need to do is upload these to the YOLOv5 datasets directory and then specify the name/path below.

In [ ]:
DETECTION_FILES = '' # e.g. 'test_video.mp4' OR test_image_directory OR test_image.jpg
CONFIDENCE_THRESHOLD = 0.50 # this should be between 0 and 1. It changes the cutoff value for a detection. Lower = more sensitive, higher = less sensitive

!python detect.py --source datasets/{DETECTION_FILES} --weights runs/train/{RUN_NAME}/weights/best.pt --name {RUN_NAME} --img {IMAGE_SIZE} --conf-thres 0.50